## Task 3: Building a database of summary tables from the Wedge transaction data.

In [1]:
import sqlite3
import os
import re
import datetime 
from zipfile import ZipFile
import pandas as pd
import numpy as np
import pandas_gbq
import janitor
import shutil
import glob
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
# Building the private key.
service_path = "C:\\Users\\rsmcd\\OneDrive\\Desktop\\MSBA Fall 2022\\" # Path to json file.
service_file = 'reese-msba-9558fdd20984.json' # Name of json file.
gbq_proj_id = 'reese-msba' # Name of project.

# Creates single variable that leads to json file.
private_key =service_path + service_file  

In [3]:
# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

In [4]:
# And finally we establish our connection.
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [5]:
# Look at list of data sets in client.
for item in client.list_datasets() : 
    print(item.full_dataset_id)

reese-msba:dram_shop
reese-msba:wedge_transactions


The following few cells access the Wedge data in GBQ to build the tables that will be used to build the dataframe.

In [6]:
# Builds a table with date, hour, total spend, number of transactions, and number of items.
query = """
SELECT EXTRACT(DATE FROM datetime) AS date
, EXTRACT(HOUR FROM datetime) AS hour
, ROUND(SUM(total), 2) AS spend 
, COUNT(DISTINCT CONCAT(CAST(EXTRACT(DATE FROM datetime) AS STRING), CAST(register_no AS STRING), CAST(emp_no AS STRING), CAST(trans_no AS STRING))) AS trans
, SUM(CASE trans_status WHEN "V" THEN -1 WHEN "R" THEN -1 ELSE 1 END) AS items
FROM `reese-msba.wedge_transactions.transArchive*` 
WHERE (department != 0
AND department != 15)
AND (trans_status IS NULL
OR trans_status = "V"
OR trans_status = "R"
OR trans_status = '')
GROUP BY date, hour
ORDER BY date DESC, hour DESC
"""

date_hour_spend = pandas_gbq.read_gbq(query,project_id = gbq_proj_id)

Downloading: 100%|██████████████████████████████████████████████████████████| 32562/32562 [00:02<00:00, 13658.11rows/s]


In [7]:
date_hour_spend

,date,hour,spend,trans,items
0,2017-01-31,22,518.25,1,165
1,2017-01-31,21,43655.85,115,12495
2,2017-01-31,20,69527.40,160,18540
3,2017-01-31,19,80792.40,208,20670
4,2017-01-31,18,104887.80,255,25695
...,...,...,...,...,...
32557,2010-01-01,13,4654.52,154,1365
32558,2010-01-01,12,3886.51,124,1143
32559,2010-01-01,11,4001.66,118,1108
32560,2010-01-01,10,3128.55,82,913


In [8]:
# Builds a table with card_no, year, month, sales, number of transactions, and number of items.
query = """
SELECT card_no
, EXTRACT(YEAR FROM datetime) AS year
, EXTRACT(MONTH FROM datetime) AS month
, ROUND(SUM(total), 2) AS sales 
, COUNT(DISTINCT CONCAT(CAST(EXTRACT(DATE FROM datetime) AS STRING), CAST(register_no AS STRING), CAST(emp_no AS STRING), CAST(trans_no AS STRING))) AS trans
, SUM(CASE trans_status WHEN "V" THEN -1 WHEN "R" THEN -1 ELSE 1 END) AS items
FROM `reese-msba.wedge_transactions.transArchive*` 
WHERE (department != 0
AND department != 15)
AND (trans_status IS NULL
OR trans_status = "V"
OR trans_status = "R"
OR trans_status = '')
GROUP BY card_no, year, month 
ORDER BY year DESC, month DESC
"""

owner_spend = pandas_gbq.read_gbq(query,project_id = gbq_proj_id)

Downloading: 100%|████████████████████████████████████████████████████████| 676786/676786 [00:34<00:00, 19672.76rows/s]


In [9]:
owner_spend

,card_no,year,month,sales,trans,items
0,13173.0,2017,1,5215.05,15,1125
1,3.0,2017,1,7034269.95,25029,1816980
2,49856.0,2017,1,313.50,2,105
3,14496.0,2017,1,1695.45,3,510
4,64761.0,2017,1,12572.25,47,3060
...,...,...,...,...,...,...
676781,12390.0,2010,1,21.52,1,4
676782,15050.0,2010,1,31.29,1,5
676783,21184.0,2010,1,17.94,1,3
676784,15475.0,2010,1,9.97,1,3


In [10]:
# Builds a table with upc, description, department number, department name, year, month, sales, transactions, and items.
query = """
SELECT rmsba.upc
, rmsba.description
, rmsba.department
, umsba.dept_name
, EXTRACT(YEAR FROM rmsba.datetime) AS year
, EXTRACT(MONTH FROM rmsba.datetime) AS month
, ROUND(SUM(rmsba.total), 2) AS sales 
, COUNT(DISTINCT CONCAT(CAST(EXTRACT(DATE FROM datetime) AS STRING), CAST(rmsba.register_no AS STRING), CAST(rmsba.emp_no AS STRING), CAST(rmsba.trans_no AS STRING))) AS trans
, SUM(CASE rmsba.trans_status WHEN "V" THEN -1 WHEN "R" THEN -1 ELSE 1 END) AS items
FROM `reese-msba.wedge_transactions.transArchive*` as rmsba
INNER JOIN `umt-msba.wedge_transactions.department_lookup` as umsba
ON rmsba.department = umsba.department
WHERE (rmsba.department != 0
AND rmsba.department != 15)
AND (rmsba.trans_status IS NULL
OR rmsba.trans_status = "V"
OR rmsba.trans_status = "R"
OR rmsba.trans_status = '')
GROUP BY rmsba.upc, rmsba.description, rmsba.department, dept_name, year, month
ORDER BY year DESC, month DESC
"""

dept_spend = pandas_gbq.read_gbq(query,project_id = gbq_proj_id)

Downloading: 100%|██████████████████████████████████████████████████████| 1001232/1001232 [01:18<00:00, 12725.70rows/s]


In [11]:
dept_spend

,upc,description,department,dept_name,year,month,sales,trans,items
0,0003320001110,Baking Soda 16oz A&H,1.0,PACKAGED GROCERY,2017,1,2212.65,128,2205
1,0001299311202,Peach Pear Spk.Water 12pk LaCr,1.0,PACKAGED GROCERY,2017,1,7535.70,99,1530
2,0020132200000,Toasted Garbanzo Salad,8.0,DELI,2017,1,10453.50,183,2955
3,0081989801064,Chkn.Wild Rice Soup 17.4oz BtN,1.0,PACKAGED GROCERY,2017,1,897.75,13,225
4,0065474935100,Lavender Shampoo 11oz Ava,11.0,PERSONAL CARE,2017,1,1467.75,14,225
...,...,...,...,...,...,...,...,...,...
1001227,0065264200090,O.Peach Spread 11.64oz Rig,1.0,PACKAGED GROCERY,2010,1,42.21,9,9
1001228,0007572000061,Nat.Spring Water 1L PS,1.0,PACKAGED GROCERY,2010,1,134.64,119,135
1001229,0020704900000,Clear Wide Mouth Jar 1oz,9.0,GEN MERCH,2010,1,147.06,55,72
1001230,0007260974183,O.Mint Chip IceCream 48oz Ald,6.0,FROZEN,2010,1,204.29,30,31


The next cell exports the tables to csv.

In [12]:
date_hour_spend.to_csv("date_hour_spend.csv", index=False, sep ='\t')
owner_spend.to_csv("owner_spend.csv", index=False, sep ='\t')
dept_spend.to_csv("dept_spend.csv", index=False, sep ='\t')

This cell creates the data base into which the tables will be read.

In [13]:
# Create connection
conn = sqlite3.connect('wedge_summary.db')

# Create cursor
curs = conn.cursor()

# Drop table if they exist to start fresh
curs.execute("DROP TABLE IF EXISTS date_hour_spend")
curs.execute("DROP TABLE IF EXISTS owner_spend")
curs.execute("DROP TABLE IF EXISTS dept_spend")

The next cell creates tables in the database that correspond in name and data type to the csv files that will be read in.

In [14]:
curs.execute("""CREATE TABLE date_hour_spend(
            date text,
            hour integer,
            spend numeric,
            trans integer,
            items integer
)""")
curs.execute("""CREATE TABLE owner_spend(
            card_no text,
            year integer,
            month integer,
            sales numeric,
            trans integer,
            items integer
)""")
curs.execute("""CREATE TABLE dept_spend(
            upc text,
            description text,
            department text,
            dept_name text,
            year integer,
            month integer,
            sales numeric,
            trans integer,
            items integer
)""")

Next we prepare the csv data to be inserted into the database tables.

In [15]:
date_hour = []

with open("date_hour_spend.csv",'r') as infile : 
    next(infile)
 
    for line in infile :
        line = line.strip().split("\t")
        date_hour.append(line)

owner = []        
        
with open("owner_spend.csv",'r') as infile : 
    next(infile)
 
    for line in infile :
        line = line.strip().split("\t")
        owner.append(line)
        
dept = []        
        
with open("dept_spend.csv",'r') as infile : 
    next(infile)
 
    for line in infile :
        line = line.strip().split("\t")
        dept.append(line)        

The next three cells insert the data into the database tables.

In [16]:
curs.executemany('''INSERT INTO date_hour_spend (date, hour, spend, trans, items) 
                   VALUES(?,?,?,?,?)''', date_hour)
conn.commit()

In [17]:
curs.executemany('''INSERT INTO owner_spend (card_no, year, month, sales, trans, items) 
                   VALUES (?,?,?,?,?,?)''', owner)
conn.commit()

In [18]:
curs.executemany('''INSERT INTO dept_spend (upc, description, department, dept_name, year, month, sales, trans, items) 
                   VALUES (?,?,?,?,?,?,?,?,?)''', dept)
conn.commit()

These last few cells query the database to make sure that everything is in working order, and then we close the database connection.

In [19]:
curs.execute("""SELECT *
            FROM date_hour_spend
            LIMIT 10""")

print(curs.fetchall())

[('2017-01-31', 22, 518.25, 1, 165), ('2017-01-31', 21, 43655.85, 115, 12495), ('2017-01-31', 20, 69527.4, 160, 18540), ('2017-01-31', 19, 80792.4, 208, 20670), ('2017-01-31', 18, 104887.8, 255, 25695), ('2017-01-31', 17, 89824.65, 245, 23100), ('2017-01-31', 16, 86167.05, 199, 21645), ('2017-01-31', 15, 87650.25, 207, 21900), ('2017-01-31', 14, 83348.25, 185, 21240), ('2017-01-31', 13, 80836.95, 214, 20910)]


In [20]:
curs.execute("""SELECT *
            FROM owner_spend
            LIMIT 10""")

print(curs.fetchall())

[('13173.0', 2017, 1, 5215.05, 15, 1125), ('3.0', 2017, 1, 7034269.95, 25029, 1816980), ('49856.0', 2017, 1, 313.5, 2, 105), ('14496.0', 2017, 1, 1695.45, 3, 510), ('64761.0', 2017, 1, 12572.25, 47, 3060), ('16743.0', 2017, 1, 7021.05, 18, 2070), ('22214.0', 2017, 1, 5131.95, 17, 2130), ('19898.0', 2017, 1, 4552.8, 21, 1215), ('11439.0', 2017, 1, 1391.4, 6, 510), ('13511.0', 2017, 1, 4924.5, 3, 1155)]


In [21]:
curs.execute("""SELECT *
            FROM dept_spend
            LIMIT 10""")

print(curs.fetchall())

[('0003320001110', 'Baking Soda 16oz A&H', '1.0', 'PACKAGED GROCERY', 2017, 1, 2212.65, 128, 2205), ('0001299311202', 'Peach Pear Spk.Water 12pk LaCr', '1.0', 'PACKAGED GROCERY', 2017, 1, 7535.7, 99, 1530), ('0020132200000', 'Toasted Garbanzo Salad', '8.0', 'DELI', 2017, 1, 10453.5, 183, 2955), ('0081989801064', 'Chkn.Wild Rice Soup 17.4oz BtN', '1.0', 'PACKAGED GROCERY', 2017, 1, 897.75, 13, 225), ('0065474935100', 'Lavender Shampoo 11oz Ava', '11.0', 'PERSONAL CARE', 2017, 1, 1467.75, 14, 225), ('0007507010447', 'BlueberryPom.Cereal 12oz Peace', '1.0', 'PACKAGED GROCERY', 2017, 1, 884.25, 16, 255), ('0007732077515', 'Bloom SPF8 Lip Balm 0.15oz', '11.0', 'PERSONAL CARE', 2017, 1, 418.95, 6, 105), ('0079357376456', 'Strawb.Lem.Kombucha 16oz Deane', '4.0', 'REF GROCERY', 2017, 1, 1241.55, 20, 345), ('0081833601023', 'Cod Pate Cat Food 3oz ILY', '1.0', 'PACKAGED GROCERY', 2017, 1, 190.8, 8, 120), ('0000000004208', 'HH WINE Glass Tiamo Pinot Grig', '22.0', 'BEER & WINE', 2017, 1, 360, 4, 

In [22]:
# Close connection to database
conn.close()